## Reading files and merge

In [84]:
import re
import nltk
import string
import numpy as np
import pandas as pd
from sklearn import svm
from nltk.tag import pos_tag
from collections import Counter
from nltk.corpus import stopwords
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn import model_selection
from nltk.stem import PorterStemmer
from sklearn.metrics import f1_score
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
from sklearn.metrics import hamming_loss
from IPython.display import display, HTML
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('tagsets')
# nltk.download('averaged_perceptron_tagger')

In [85]:
file=pd.read_csv("Questions.csv",encoding="ISO-8859-1")
file2=pd.read_csv("Tags.csv",encoding="ISO-8859-1")

In [86]:
file.drop(columns=['OwnerUserId','CreationDate','ClosedDate'],inplace=True)

In [87]:
# file.head()

In [88]:
# file2.head()

## merge two files

In [89]:
merged_df = file2.groupby('Id')['Tag'].agg(list).reset_index()
        
merged_df

data=pd.merge(file,merged_df,on='Id')

In [90]:
data

,Id,Score,Title,Body,Tag
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]"
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net, sitemap]"
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors, color-s..."
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]"
...,...,...,...,...,...
1264211,40143210,0,URL routing in PHP (MVC),<p>I am building a custom MVC project and I ha...,"[php, .htaccess]"
1264212,40143300,0,Bigquery.Jobs.Insert - Resumable Upload?,<p>The API docs show that you should be able t...,[google-bigquery]
1264213,40143340,1,Obfuscating code in android studio,<p>Under minifyEnabled I changed from false to...,"[android, android-studio]"
1264214,40143360,0,How to fire function after v-model change?,<p>I have input which I use to filter my array...,"[javascript, vue.js]"


## Preprocessing

##### we filtered the data that has high score to reduce amount of the data as it was a large data set (>10^6 row)

In [91]:
data=data[data['Score']>10]

In [92]:
data

,Id,Score,Title,Body,Tag
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]"
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net, sitemap]"
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors, color-s..."
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]"
...,...,...,...,...,...
1256569,39963870,11,Is using Single as an Assert a bad practice?,<p>I'm testing a method that manipulates a col...,"[c#, linq, unit-testing]"
1257356,39981800,11,How to achieve test isolation with Symfony for...,<p><strong><em>Note:</strong> This is Symfony ...,"[php, unit-testing, symfony2, symfony-forms]"
1261242,40073530,16,Why doesn't ${@:-1} return the last element of...,<p><em>I thought to post up a Q&amp;A on this ...,[bash]
1262332,40098470,19,If multiple classes have a static variable in ...,<p>I have the following example code:</p>\n\n<...,"[c++, static, static-members]"


In [93]:
data.to_csv('NEW_DATA.csv',index=False)
data=pd.read_csv('NEW_DATA.csv',encoding="ISO-8859-1")

In [94]:
data.drop(columns=['Score','Id'],inplace=True)

In [95]:
def to_list(s):
    return s[1:-1].replace("'", "").split(',')
data['Tag'] = data['Tag'].apply(to_list)

In [96]:
# Define a function to remove leading spaces from each string in the array
def remove_spaces(arr):
    return [s.lstrip() for s in arr]

# Apply the function to the 'array_col' column
data['Tag'] = data['Tag'].apply(remove_spaces)

###### get the most freq tags

In [97]:
Freq_tags = set()
# Flatten the list of keywords into a single list of words
words = [tag for row in data['Tag'] for tag in row ]
# # Get the frequency count of each word
freq_count = Counter(words)
# stop_words = set(stopwords.words('english'))

# Print the 10 most common words
for word, count in freq_count.most_common(80):
        Freq_tags.add(word)
print(Freq_tags)

{'xml', 'json', 'winforms', 'oracle', 'ios', 'math', 'performance', 'haskell', 'asp.net-mvc-3', 'c#', 'node.js', 'ajax', 'c', 'html5', 'postgresql', 'oop', 'ruby-on-rails', 'github', 'osx', 'google-chrome', 'css3', 'entity-framework', 'perl', 'iphone', 'debugging', 'http', 'ruby-on-rails-3', 'algorithm', 'twitter-bootstrap', 'bash', 'linux', 'regex', 'database', 'list', 'asp.net-mvc', 'svn', 'javascript', 'r', 'visual-studio', 'swift', 'jquery', 'css', 'windows', 'ruby', 'android', 'gcc', 'eclipse', 'cocoa', 'tsql', 'objective-c', 'asp.net', 'mongodb', 'django', 'arrays', 'mysql', 'html', 'maven', 'cocoa-touch', 'string', '.net', 'multithreading', 'xcode', 'wpf', 'linq', 'php', 'vim', 'python', 'java', 'spring', 'generics', 'angularjs', 'sql-server', 'shell', 'git', 'c++', 'sql', 'visual-studio-2010', 'unit-testing', 'scala', 'c++11'}


In [98]:
# (6) Display Words Occurrences By Changing Font Size Using HTML
html_str=''
for word in freq_count.keys():
    font_size = freq_count[word] / 50
    html_str+='<span style="font-size:{}px">{}</span> '.format(font_size, word)
display(HTML(html_str))

In [99]:
data['Tag']

0                              [flex, actionscript-3, air]
1        [svn, tortoisesvn, branch, branching-and-merging]
2                                  [sql, asp.net, sitemap]
3        [algorithm, language-agnostic, colors, color-s...
4             [c#, .net, scripting, compiler-construction]
                               ...                        
32599                             [c#, linq, unit-testing]
32600         [php, unit-testing, symfony2, symfony-forms]
32601                                               [bash]
32602                        [c++, static, static-members]
32603    [c++, lambda, c++14, language-lawyer, member-i...
Name: Tag, Length: 32604, dtype: object

#### Remove the unique tags from the data frame

In [100]:
def removeUniqueTags(lst):
    for tag in lst:
        if(tag not in Freq_tags):
            lst.remove(tag)
    return lst

data['Tag']=data['Tag'].apply(removeUniqueTags)

In [101]:
data['Tag']

0                           [actionscript-3]
1                              [svn, branch]
2                             [sql, asp.net]
3                        [algorithm, colors]
4          [c#, .net, compiler-construction]
                        ...                 
32599               [c#, linq, unit-testing]
32600     [php, unit-testing, symfony-forms]
32601                                 [bash]
32602                  [c++, static-members]
32603    [c++, c++14, member-initialization]
Name: Tag, Length: 32604, dtype: object

In [102]:
data['Tag'] = data['Tag'].apply(lambda x: None if len(x)==0 else x)

In [103]:
data['Tag']

0                           [actionscript-3]
1                              [svn, branch]
2                             [sql, asp.net]
3                        [algorithm, colors]
4          [c#, .net, compiler-construction]
                        ...                 
32599               [c#, linq, unit-testing]
32600     [php, unit-testing, symfony-forms]
32601                                 [bash]
32602                  [c++, static-members]
32603    [c++, c++14, member-initialization]
Name: Tag, Length: 32604, dtype: object

In [104]:
data.dropna(inplace=True)
data

,Title,Body,Tag
0,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,[actionscript-3]
1,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, branch]"
2,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net]"
3,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, colors]"
4,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, compiler-construction]"
...,...,...,...
32599,Is using Single as an Assert a bad practice?,<p>I'm testing a method that manipulates a col...,"[c#, linq, unit-testing]"
32600,How to achieve test isolation with Symfony for...,<p><strong><em>Note:</strong> This is Symfony ...,"[php, unit-testing, symfony-forms]"
32601,Why doesn't ${@:-1} return the last element of...,<p><em>I thought to post up a Q&amp;A on this ...,[bash]
32602,If multiple classes have a static variable in ...,<p>I have the following example code:</p>\n\n<...,"[c++, static-members]"


## Preprocessing Title and Body

### 1) Body

In [105]:
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

### Tokenize

In [106]:
def Tokenize(txt):
    return word_tokenize(txt)

### Remove Stop_Words

In [107]:
tag_dict = {"J": wn.ADJ,
            "N": wn.NOUN,
            "V": wn.VERB,
            "R": wn.ADV}

In [108]:
stop_words=set(stopwords.words("english"))

def remove_stopWords(ColName):
    for i,Row in data.iterrows():
        filtered_list=[]
        for word in Row[ColName]:
            if word.casefold() not in stop_words:
                filtered_list.append(word)        
        data.loc[i,ColName]=filtered_list
        

def remove_punctuatuion(ColName):
    for i,Row in data.iterrows():
        no_punct_filtered_speech_words=[''.join(char for char in word if char not in string.punctuation)
                                   for word in Row[ColName] ]
        no_punct_filtered_speech_words=[word for word in no_punct_filtered_speech_words if word]
        data.loc[i,ColName]=no_punct_filtered_speech_words

def Lemmatizing(ColName):
    lemmatizer=WordNetLemmatizer()
    for i,Row in data.iterrows():
        Lemmatized_words=[]
        for word in Row[ColName]:
#             tag = nltk.pos_tag([word])[0][1][0].upper()
            Lemmatized_words.append(lemmatizer.lemmatize(word.lower(),pos='v'))
        data.loc[i,ColName]=Lemmatized_words
    
def Stemming(ColName):
    stemmer = PorterStemmer()
    for i,Row in data.iterrows():
        stemmed_words=[]
        for word in Row[ColName]:
            stemmed_words.append(stemmer.stem(word))
        data.loc[i,ColName]=stemmed_words

## 1)Body

In [109]:
data["Body"] = data["Body"].apply(remove_html_tags)

In [110]:
data['Body']=data['Body'].apply(Tokenize)

In [111]:
remove_stopWords('Body')

In [112]:
Stemming('Body')

In [113]:
# Lemmatizing('Body')

In [114]:
remove_punctuatuion('Body')

In [115]:
data["Body"]

0        [ve, written, databas, gener, script, sql, wan...
1        [realli, good, tutori, explain, branch, merg, ...
2        [anyon, got, experi, creat, sqlbase, aspnet, s...
3        [someth, ve, pseudosolv, mani, time, never, qu...
4        [littl, game, written, C, use, databas, backen...
                               ...                        
32599    [m, test, method, manipul, collect, given, set...
32600    [note, symfoni, lt, 26, believ, overal, issu, ...
32601    [thought, post, Q, amp, find, anyth, similar, ...
32602    [follow, exampl, code, class, public, static, ...
32603    [includ, lt, cassert, gt, int, main, struct, p...
Name: Body, Length: 31515, dtype: object

### 2) Title

In [116]:
data['Title']=data['Title'].apply(Tokenize)

In [117]:
remove_stopWords('Title')

In [118]:
Stemming('Title')

In [119]:
# Lemmatizing('Title')

In [120]:
remove_punctuatuion('Title')

In [121]:
data["Title"]

0        [sqlstatementexecut, multipl, queri, one, stat...
1                [good, branch, merg, tutori, tortoisesvn]
2                                      [aspnet, site, map]
3                          [function, creat, color, wheel]
4                      [ad, script, function, net, applic]
                               ...                        
32599                   [use, singl, assert, bad, practic]
32600    [achiev, test, isol, symfoni, form, data, tran...
32601                       [nt, 1, return, last, element]
32602    [multipl, class, static, variabl, common, shar...
32603        [use, lambda, default, initi, gcc, vs, clang]
Name: Title, Length: 31515, dtype: object

## TF_IDF

In [122]:
#Splitting data into train and validation
# convert the list of words to a string
data['Title'] = data['Title'].apply(lambda x: ' '.join(x))
data['Body'] = data['Body'].apply(lambda x: ' '.join(x))

tfidf_vect_Title = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0005,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       ngram_range = (1, 3),
                                       preprocessor=None,
                                       max_features=32000)

tfidf_vect_Body = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       ngram_range = (1, 3),
                                       preprocessor=None,
                                       max_features=32000)

tfidf_Title = tfidf_vect_Title.fit_transform(data['Title'])

tfidf_Body = tfidf_vect_Body.fit_transform(data['Body'])

In [123]:
from scipy.sparse import hstack
data_tfidf = hstack([tfidf_Title,tfidf_Body])
train_x, valid_x, train_y, valid_y = train_test_split(data_tfidf, data['Tag'],random_state=23)

In [124]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    return metrics.accuracy_score(predictions, valid_y)

In [125]:
from sklearn.preprocessing import MultiLabelBinarizer

# Convert the list of tags for each sample to a binary array
multilabel_binarizer = MultiLabelBinarizer()
train_y = multilabel_binarizer.fit_transform(train_y)
valid_y = multilabel_binarizer.transform(valid_y)

# Convert train_y and valid_y from binary arrays to 1-dimensional arrays
train_y = np.argmax(train_y, axis=1)
valid_y = np.argmax(valid_y, axis=1)


In [126]:
def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Accuracy score: {}".format(accuracy_score(valid_y, y_pred)))
    print("Recall score: {}".format(recall_score(y_true=valid_y, y_pred=y_pred, average='weighted')))
    print("Precision score: {}".format(precision_score(y_true=valid_y, y_pred=y_pred, average='weighted')))
    print("Hamming loss: {}".format(hamming_loss(y_pred, valid_y)*100))
    print("F1 score: {}".format(f1_score(y_pred, valid_y, average='weighted')))
    print("---")    

In [128]:
from sklearn.metrics import confusion_matrix
svc1 = LinearSVC(C=0.09,penalty='l2')
svc1.fit(train_x, train_y)
y_pred = svc1.predict(valid_x)
print_score(y_pred, valid_y)
print (confusion_matrix(valid_y, y_pred))

Clf:  ndarray
Accuracy score: 0.49435207513643864
Recall score: 0.49435207513643864
Precision score: 0.3978392971659957
Hamming loss: 50.56479248635613
F1 score: 0.5580045728674308
---
[[  0   0   8 ...   0   0   0]
 [  0   3   0 ...   0   0   0]
 [  0   0 119 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]


In [129]:
sgd = SGDClassifier(n_jobs=-1)
sgd.fit(train_x, train_y)
y_pred = sgd.predict(valid_x)
print_score(y_pred, valid_y)
print (confusion_matrix(valid_y, y_pred))

Clf:  ndarray
Accuracy score: 0.46820662520624445
Recall score: 0.46820662520624445
Precision score: 0.39639596127880655
Hamming loss: 53.179337479375555
F1 score: 0.5145822543567555
---
[[  0   0   5 ...   0   0   0]
 [  0   1   0 ...   0   0   0]
 [  0   0 109 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
